## Preambule

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from tqdm import tqdm

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Input

In [2]:
rules = ['GF', 'PC', 'PCC', 'AP', 'GDR', 'ECPC']
path_data = "K:/ECEMF/T5.2/"

## Read data

In [4]:
xr_dataread = xr.open_dataset(path_data+"xr_dataread.nc")
xr_temps = xr.open_dataset(path_data+"xr_temps.nc")
xr_2030 = xr.open_dataset(path_data+"xr_alloc_2030.nc")

FileNotFoundError: [Errno 2] No such file or directory: b'K:\\ECEMF\\T5.2\\xr_temps.nc'

In [ ]:
df_eng = pd.read_csv('X:/user/dekkerm/Data/ENGAGE/PolicyScenarios/engage-internal_snapshot_1695642798.csv')
df_eng = df_eng[df_eng.Variable == 'Emissions|CO2']
df_eng = df_eng.reset_index(drop=True)
regions_df = np.array(df_eng.Region)
regions_df[regions_df == "Argentine Republic"] = 'ARG'
regions_df[regions_df == "Canada"] = 'CAN'
regions_df[regions_df == "Commonwealth of Australia"] = 'AUS'
regions_df[regions_df == "Federative Republic of Brazil"] = 'BRA'
regions_df[regions_df == "People's Repulic of China"] = 'CHN'
regions_df[regions_df == "European Union (28 member countries)"] = 'EU'
regions_df[regions_df == "Republic of India"] = 'IND'
regions_df[regions_df == "Republic of Indonesia"] = 'IDN'
regions_df[regions_df == "State of Japan"] = 'JPN'
regions_df[regions_df == "Russian Federation"] = 'RUS'
regions_df[regions_df == "Kingdom of Saudi Arabia"] = 'SAU'
regions_df[regions_df == "Republic of South Africa"] = 'ZAF'
regions_df[regions_df == "Republic of Korea (South Korea)"] = 'KOR'
regions_df[regions_df == "United Mexican States"] = 'MEX'
regions_df[regions_df == "Republic of Turkey"] = 'TUR'
regions_df[regions_df == "United States of America"] = 'USA'
regions_df[regions_df == "Viet Nam "] = 'VNM'
df_eng.Region = regions_df
df = pd.read_excel("X:/user/dekkerm/Data//UNFCCC_Parties_Groups_noeu.xlsx", sheet_name = "Country groups")
countries_iso = np.array(df["Country ISO Code"])
group_eu = countries_iso[np.array(df["EU"]) == 1]

In [ ]:
curpol = "GP_CurPol_T45"
ndc = "GP_NDC2030_T45"
nz = "GP_Glasgow"
df_eng_ref = df_eng[['Model', 'Scenario', 'Region']+list(df_eng.keys()[5:])]
df_eng_ref = df_eng_ref[df_eng_ref.Scenario.isin([curpol, ndc, nz])]
scen = np.array(df_eng_ref.Scenario)
scen[scen == ndc] = 'NDC'
scen[scen == curpol] = 'CurPol'
scen[scen == nz] = 'NetZero'
reg = np.array(df_eng_ref.Region)
reg[reg == 'World'] = 'WORLD'
df_eng_ref['Scenario'] = scen
df_eng_ref['Region'] = reg
dummy = df_eng_ref.melt(id_vars=["Scenario", "Model", "Region"], var_name="Time", value_name="Value")
dummy['Time'] = np.array(dummy['Time'].astype(int))
dummy = dummy.set_index(["Scenario", "Model", "Region", "Time"])
xr_eng = xr.Dataset.from_dataframe(dummy)
xr_eng = xr_eng.reindex(Time = np.arange(1850, 2101))
xr_eng = xr_eng.interpolate_na(dim="Time", method="linear")

Cost optimal

In [ ]:
df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_ISO3_v1.1.csv")
df_ar6 = df_ar6[df_ar6.Variable.isin(['Emissions|CO2', 'Policy Cost|Consumption Loss'])]
df_ar6 = df_ar6.reset_index(drop=True)

df_ar6_meta = pd.read_excel("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx", sheet_name='meta_Ch3vetted_withclimate')
mods = np.array(df_ar6_meta.Model)
scens = np.array(df_ar6_meta.Scenario)
modscens_meta = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6_meta['ModelScenario'] = modscens_meta
df_ar6_meta = df_ar6_meta[['ModelScenario', 'Category', 'Policy_category']]
df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(['C1', 'C2'])]
df_ar6_meta = df_ar6_meta.reset_index(drop=True)
ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

mods = np.array(df_ar6.Model)
scens = np.array(df_ar6.Scenario)
modscens = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6['ModelScenario'] = modscens
df_ar6 = df_ar6.drop(['Model', 'Scenario', 'Unit'], axis=1)
df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

dummy = df_ar6.melt(id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value")
dummy['Time'] = np.array(dummy['Time'].astype(int))
dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
xr_scen_r = xr.Dataset.from_dataframe(dummy)
xr_scen_r = xr_scen_r.reindex(Time = np.arange(1850, 2101))
xr_scen_r = xr_scen_r.interpolate_na(dim="Time", method="linear")

In [ ]:
df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_ISO3_v1.1.csv")
df_ar6 = df_ar6[df_ar6.Variable.isin(['Emissions|CO2', 'Policy Cost|Consumption Loss'])]
df_ar6 = df_ar6.reset_index(drop=True)

df_ar6_meta = pd.read_excel("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx", sheet_name='meta_Ch3vetted_withclimate')
mods = np.array(df_ar6_meta.Model)
scens = np.array(df_ar6_meta.Scenario)
modscens_meta = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6_meta['ModelScenario'] = modscens_meta
df_ar6_meta = df_ar6_meta[['ModelScenario', 'Category', 'Policy_category']]
df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(['C3'])]
df_ar6_meta = df_ar6_meta.reset_index(drop=True)
ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

mods = np.array(df_ar6.Model)
scens = np.array(df_ar6.Scenario)
modscens = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
df_ar6['ModelScenario'] = modscens
df_ar6 = df_ar6.drop(['Model', 'Scenario', 'Unit'], axis=1)
df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

dummy = df_ar6.melt(id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value")
dummy['Time'] = np.array(dummy['Time'].astype(int))
dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
xr_scen_r2 = xr.Dataset.from_dataframe(dummy)
xr_scen_r2 = xr_scen_r2.reindex(Time = np.arange(1850, 2101))
xr_scen_r2 = xr_scen_r2.interpolate_na(dim="Time", method="linear")

## Plot

In [ ]:
rulecolors = ['sienna', 'goldenrod', 'forestgreen', 'mediumvioletred', 'steelblue', 'tomato', 'silver']
rules = ['GF', 'PC', 'PCC', 'ECPC', 'AP', 'GDR']
standard_settings = {"TrajUnc":"Medium",
                     "NonCO2":0.5,
                     "NegEmis":0.5,
                     "Convergence_year":2050}

In [ ]:
for REG in ['EU']:
    xr_nld = xr.open_dataset(path_data+"Allocations/xr_alloc_"+REG+".nc")
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=["<b>Allocations in 2030<br>(absolute)", "<b>Emission reductions<br>(2030 compared to 2015)"],
                        horizontal_spacing = 0.07, vertical_spacing=0.06)    
    # 2030 allocations
    fig.add_traces(go.Scatter(x=['NDC'],
                            y=[xr_dataread.CO2_ndc.sel(Region=REG).sel(Time=2030).max()],
                            name='NDCs',
                            line={'color': 'black', 'width': 3},
                            mode='lines+markers',
                            marker={'size': 17, 'symbol': 'triangle-down'},
                            showlegend=False), 1, 1)
    fig.add_traces(go.Scatter(x=['NDC'],
                            y=[xr_dataread.CO2_ndc.sel(Region=REG).sel(Time=2030).min()],
                            name='NDCs',
                            line={'color': 'black', 'width': 3},
                            mode='lines+markers',
                            marker={'size': 17, 'symbol': 'triangle-up'},
                            showlegend=False), 1, 1)
    fig.add_traces(go.Scatter(x=['NDC', 'NDC'],
                            y=[xr_dataread.CO2_ndc.sel(Region=REG).sel(Time=2030).min(),
                            xr_dataread.CO2_ndc.sel(Region=REG).sel(Time=2030).max()],
                            name='NDCs',
                            line={'color': 'black', 'width': 3},
                            mode='lines',
                            marker={'size': 11, 'symbol': 'triangle-up'},
                            showlegend=False), 1, 1)
    try:
        fig.add_traces(go.Scatter(x=['Current<br>Policies']*len(xr_eng.Model),
                                y=xr_eng.sel(Time=2030, Region=REG, Scenario="CurPol").Value,
                                name='Current policies',
                                mode='lines+markers',
                                marker_color='silver',
                                marker_line_color='black',
                                marker_line_width=1,
                                marker={'size': 9, 'symbol': 'square'},
                                showlegend=False),
                        1, 1
                        )
        vals = np.array(xr_scen_r.sel(Time=2030, Region=REG, Variable="Emissions|CO2").Value)
        fig.add_traces(go.Violin(x=["Cost<br>optimal"]*len(vals),
                                        y=vals,
                                        showlegend=False,
                                        points=False,
                                        opacity=1.0,
                                        line={'color': 'tomato', 'width': 3},
                                        line_color='tomato'),
                                1, 1
                                )
        vals2 = np.array(xr_scen_r2.sel(Time=2030, Region=REG, Variable="Emissions|CO2").Value)
        fig.add_traces(go.Violin(x=["Cost<br>optimal"]*len(vals2),
                                        y=vals2,
                                        showlegend=False,
                                        points=False,
                                        opacity=0.5,
                                        line={'color': 'tomato', 'width': 3},
                                        line_color='tomato'),
                                1, 1
                                )
        med = np.nanmedian(vals)
        fig.add_traces(go.Scatter(x=['NDC', 'Current<br>Policies', "Cost<br>optimal"],
                                        y=[np.nanmedian(vals)]*3,
                                        line={'color': 'tomato', 'width': 2, 'dash': 'dot'},
                                        mode='lines',
                                        showlegend=False), 1, 1
                        )
        fig.add_traces(go.Scatter(x=['NDC'],
                                        y=[np.nanmedian(vals)],
                                        marker={'color': 'tomato', 'size': 15},
                                        mode='markers',
                                        showlegend=False), 1, 1
                        )
        regval = float(xr_dataread.sel(Region=REG, Time=2015).CO2_hist)
        fig.add_traces(go.Scatter(x=['NDC', 'Current<br>Policies']+list(rules[:rule_i+1]),
                                y=[regval]*(len(rules[:rule_i+1])+2),
                                line={'color': 'black', 'width': 1, 'dash': 'dash'},
                                mode='lines',
                                showlegend=False), 1, 1
                )
        fig.add_traces(go.Scatter(x=['NDC'],
                                y=[regval],
                                text='         2015<br>     emissions',
                                line={'color': 'black', 'width': 2, 'dash': 'dash'},
                                textfont={'color': 'black', 'size': 14},
                                textposition='middle right',
                                mode='text',
                                showlegend=False), 1, 1
                )
    except:
        3
    # fig.add_traces(go.Scatter(x=['NDC', 'CurPol'],
    #                         y=[np.array(xr_eng.sel(Time=2030, Region=REG, Scenario="CurPol").Value.median())]*2,
    #                         line={'color': 'silver', 'width': 2, 'dash': 'dot'},
    #                         mode='lines',
    #                         showlegend=False),
    #                 1, 1
    #                 )
    # fig.add_traces(go.Scatter(x=['NDC'],
    #                         y=[np.array(xr_eng.sel(Time=2030, Region=REG, Scenario="CurPol").Value.median())],
    #                         marker={'color': "grey", 'size': 15},
    #                         mode='markers',
    #                         showlegend=False), 1, 1
    #                 )
    for rule_i, rule in enumerate(rules):
        for T_i, T in enumerate([1.5, 2.0]):
            vals = np.array(xr_nld.sel(Temperature=T, Risk=[0.5, 0.33][T_i])[rule].sel(Time=2030)).flatten()
            vals[vals == 0] = np.nan
            if T == 1.5:
                med = np.nanmedian(vals)
                fig.add_traces(go.Scatter(x=['NDC', 'Current<br>Policies', 'Cost<br>optimal']+list(rules[:rule_i+1]),
                                    y=[np.nanmedian(vals)]*(len(rules[:rule_i+1])+3),
                                    line={'color': rulecolors[rule_i], 'width': 1, 'dash': 'dot'},
                                    mode='lines',
                                    showlegend=False), 1, 1
                        )
                fig.add_traces(go.Scatter(x=['NDC'],
                                    y=[np.nanmedian(vals)],
                                    marker={'color': rulecolors[rule_i], 'size': 15},
                                    mode='markers',
                                    showlegend=False), 1, 1
                        )
            fig.add_traces(go.Violin(x=[rule]*len(vals),
                                    y=vals,
                                    showlegend=False,
                                    points=False,
                                    opacity=1.0-T_i*0.5,
                                    line={'color': 'black', 'width': 3},
                                    line_color=rulecolors[rule_i]),
                            1, 1
                            )
    fig.add_traces(go.Scatter(x=['NDC'],
                            y=[xr_dataread.CO2_ndc.sel(Region=REG).sel(Time=2030).max()],
                            name='NDCs',
                            line={'color': 'black', 'width': 3},
                            mode='lines+markers',
                            marker={'size': 17, 'symbol': 'triangle-down'},
                            showlegend=False), 1, 1)
    fig.add_traces(go.Scatter(x=['NDC'],
                            y=[xr_dataread.CO2_ndc.sel(Region=REG).sel(Time=2030).min()],
                            name='NDCs',
                            line={'color': 'black', 'width': 3},
                            mode='lines+markers',
                            marker={'size': 17, 'symbol': 'triangle-up'},
                            showlegend=False), 1, 1)

    # Reductions in 2030 w.r.t. 2021
    regval = float(xr_dataread.sel(Region=REG, Time=2015).CO2_hist)
    fig.add_traces(go.Scatter(x=['NDC'],
                            y=[(xr_dataread.CO2_ndc.sel(Region=REG).sel(Time=2030).max()-regval)/regval*-100],
                            name='NDCs',
                            line={'color': 'black', 'width': 3},
                            mode='lines+markers',
                            marker={'size': 17, 'symbol': 'triangle-up'},
                            showlegend=False),  1, 2)
    fig.add_traces(go.Scatter(x=['NDC'],
                            y=[(xr_dataread.CO2_ndc.sel(Region=REG).sel(Time=2030).min()-regval)/regval*-100],
                            name='NDCs',
                            line={'color': 'black', 'width': 3},
                            mode='lines+markers',
                            marker={'size': 17, 'symbol': 'triangle-down'},
                            showlegend=False),  1, 2)
    fig.add_traces(go.Scatter(x=['NDC', 'NDC'],
                            y=[(xr_dataread.CO2_ndc.sel(Region=REG).sel(Time=2030).min()-regval)/regval*-100,
                            (xr_dataread.CO2_ndc.sel(Region=REG).sel(Time=2030).max()-regval)/regval*-100],
                            name='NDCs',
                            line={'color': 'black', 'width': 3},
                            mode='lines',
                            marker={'size': 11, 'symbol': 'triangle-down'},
                            showlegend=False),  1, 2)
    try:
        fig.add_traces(go.Scatter(x=['Current<br>Policies']*len(xr_eng.Model),
                                y=(np.array(xr_eng.sel(Time=2030, Region=REG, Scenario="CurPol").Value) - regval)/regval*-100,
                                name='Current policies',
                                mode='lines+markers',
                                marker_color='silver',
                                marker_line_color='black',
                                marker_line_width=1,
                                marker={'size': 9, 'symbol': 'square'},
                                showlegend=False),
                        1, 2
                        )
        vals = (np.array(xr_scen_r.sel(Time=2030, Region=REG, Variable="Emissions|CO2").Value) - regval)/regval*-100
        fig.add_traces(go.Violin(x=["Cost<br>optimal"]*len(vals),
                                        y=vals,
                                        showlegend=False,
                                        points=False,
                                        opacity=1.0,
                                        line={'color': 'tomato', 'width': 3},
                                        line_color='tomato'),
                                1, 2
                                )
        vals2 = (np.array(xr_scen_r2.sel(Time=2030, Region=REG, Variable="Emissions|CO2").Value) - regval)/regval*-100
        fig.add_traces(go.Violin(x=["Cost<br>optimal"]*len(vals2),
                                        y=vals2,
                                        showlegend=False,
                                        points=False,
                                        opacity=0.5,
                                        line={'color': 'tomato', 'width': 3},
                                        line_color='tomato'),
                                1, 2
                                )
        med = np.nanmedian(vals)
        fig.add_traces(go.Scatter(x=['NDC', 'Current<br>Policies', "Cost<br>optimal"],
                                        y=[np.nanmedian(vals)]*3,
                                        line={'color': 'tomato', 'width': 1, 'dash': 'dot'},
                                        mode='lines',
                                        showlegend=False), 1, 2
                        )
        fig.add_traces(go.Scatter(x=['NDC'],
                                        y=[np.nanmedian(vals)],
                                        marker={'color': 'tomato', 'size': 15},
                                        mode='markers',
                                        showlegend=False), 1, 2
                        )
        fig.add_traces(go.Scatter(x=['NDC', 'Current<br>Policies']+list(rules[:rule_i+1]),
                                y=[0]*(len(rules[:rule_i+1])+2),
                                line={'color': 'black', 'width': 2, 'dash': 'dash'},
                                mode='lines',
                                showlegend=False), 1, 2
                )
    except:
        3
    # fig.add_traces(go.Scatter(x=['NDC', 'CurPol'],
    #                         y=[(np.array(xr_eng.sel(Time=2030, Region=REG, Scenario="CurPol").Value.median()) - regval)/regval*-100]*2,
    #                         line={'color': 'silver', 'width': 2, 'dash': 'dot'},
    #                         mode='lines',
    #                         showlegend=False),
    #                 1, 2
    #                 )
    # fig.add_traces(go.Scatter(x=['NDC'],
    #                         y=[(np.array(xr_eng.sel(Time=2030, Region=REG, Scenario="CurPol").Value.median()) - regval)/regval*-100],
    #                         marker={'color': "grey", 'size': 15},
    #                         mode='markers',
    #                         showlegend=False), 1, 2
    #                 )
    for rule_i, rule in enumerate(rules):
        for T_i, T in enumerate([1.5, 2.0]):
            vals = (np.array(xr_nld.sel(Temperature=T, Risk=[0.5, 0.33][T_i])[rule].sel(Time=2030)).flatten()-regval)/regval*-100
            vals[vals == 0] = np.nan
            if T == 1.5:
                med = np.nanmedian(vals)
                fig.add_traces(go.Scatter(x=['NDC', 'Current<br>Policies', 'Cost<br>optimal']+list(rules[:rule_i+1]),
                                    y=[np.nanmedian(vals)]*(len(rules[:rule_i+1])+3),
                                    line={'color': rulecolors[rule_i], 'width': 1, 'dash': 'dot'},
                                    mode='lines',
                                    showlegend=False), 1, 2
                        )
                fig.add_traces(go.Scatter(x=['NDC'],
                                    y=[np.nanmedian(vals)],
                                    marker={'color': rulecolors[rule_i], 'size': 15},
                                    mode='markers',
                                    showlegend=False), 1, 2
                        )
            fig.add_traces(go.Violin(x=[rule]*len(vals),
                                    y=vals,
                                    showlegend=False,
                                    points=False,
                                    opacity=1.0-T_i*0.5,
                                    line={'color': 'black', 'width': 3},
                                    line_color=rulecolors[rule_i]),
                            1, 2
                            )
    fig.add_traces(go.Scatter(x=['NDC'],
                            y=[(xr_dataread.CO2_ndc.sel(Region=REG).sel(Time=2030).max()-regval)/regval*-100],
                            name='NDCs',
                            line={'color': 'black', 'width': 3},
                            mode='lines+markers',
                            marker={'size': 17, 'symbol': 'triangle-up'},
                            showlegend=False),  1, 2)
    fig.add_traces(go.Scatter(x=['NDC'],
                            y=[(xr_dataread.CO2_ndc.sel(Region=REG).sel(Time=2030).min()-regval)/regval*-100],
                            name='NDCs',
                            line={'color': 'black', 'width': 3},
                            mode='lines+markers',
                            marker={'size': 17, 'symbol': 'triangle-down'},
                            showlegend=False),  1, 2)
    fig.update_annotations(font=dict(family="Helvetica", size=25))
    fig.update_layout(height=800, width=1900, template='simple_white', font=dict(family="Helvetica"))
    fig.update_yaxes(tickfont=dict(size=20), row=1, col=1, title='CO<sub>2</sub> emissions allocations in 2030 ('+REG+')<br>[Mt CO<sub>2</sub>]', titlefont=(dict(size=20)))
    fig.update_xaxes(tickfont=dict(size=20), row=1, col=1)
    fig.update_yaxes(tickfont=dict(size=20), row=1, col=2, title='% reduction with respect to 2015', titlefont=(dict(size=20)), range=(-150, 150), tickvals=[-150, -100, -50, 0, 50, 100, 150], ticktext=['-150%', '-100%', '-50%', '0%', '50%', '100%', '150%'])
    fig.update_xaxes(tickfont=dict(size=20), row=1, col=2)
    fig.write_image("X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Presentations/Alloc2030_"+REG+".png", scale=3)
fig.show()

3